In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import os
%matplotlib inline 

In [2]:
from AnomalousCommunityDetection.GraphSampler import GraphSampler
from AnomalousCommunityDetection.BiPartiteConverter import BiPartiteConverter
from AnomalousCommunityDetection.Utils import BiPartiteUtils
from AnomalousCommunityDetection.FeatureExtractor import FeatureExtractor
from AnomalyInfusedCommunityStructuredRandomNetworkGenerator.AnomalyInfusedCommunityStructuredRandomNetworkGenerator import AnomalyInfusedCommunityStructuredRandomNetworkGenerator

In [5]:
BiPartiteConverter()

TypeError: __init__() missing 1 required positional argument: 'partitions_map_path'

In [ ]:
from UseCases.Wikipedia.PreProcessing.WikipediaRevisionsGraphSimulator import WikipediaRevisionsGraphSimulator
from SimulatedGraphExperiment.GraphCommunityColorer import GraphCommunityColorer

In [ ]:
# path to save data-reduced df
NORM_EDGES_PATH = 'UseCases\\Wikipedia\\Data\\norm_edges.csv'
ANOM_EDGES_PATH = 'UseCases\\Wikipedia\\Data\\anom_edges.csv'

In [ ]:
# later maybe to be added - paths to real graph (not bipartite)
NORM_GRAPH_PATH = 'UseCases\\Wikipedia\\DataVisualization\\norm_graph.gml'
ANOM_GRAPH_PATH = 'UseCases\\Wikipedia\\DataVisualization\\anom_graph.gml'

NORM_BI_PARTITE_GRAPH_PATH = 'UseCases\\Wikipedia\\DataVisualization\\norm_bi_partite_graph.gml'
ANOM_BI_PARTITE_GRAPH_PATH = 'UseCases\\Wikipedia\\DataVisualization\\anom_bi_partite_graph.gml'

PROJECTED_BI_PARTITE_GRAPH_1_PATH = 'UseCases\\Wikipedia\\DataVisualization\\projected_bi_partite_graph_1.gml'
PROJECTED_BI_PARTITE_GRAPH_2_PATH = 'UseCases\\Wikipedia\\DataVisualization\\projected_bi_partite_graph_2.gml'

In [ ]:
# path to save topological features DataFrames of train and test set
EDGES_TOPOLOGICAL_FEATURES_ALL_EDGES_PATH = 'UseCases\\Wikipedia\\Data\\edges_topological_features_all_edges.csv'
EDGES_TOPOLOGICAL_FEATURES_TRAIN_SET_PATH = 'UseCases\\Wikipedia\\Data\\edges_topological_features_train_set.csv'
EDGES_TOPOLOGICAL_FEATURES_TEST_SET_PATH = 'UseCases\\Wikipedia\\Data\\edges_topological_features_test_set.csv'

In [ ]:
# path to save test set meta-features DataFrame
NODES_META_FEATURES_TEST_SET_PATH = 'UseCases\\Wikipedia\\Data\\nodes_meta_features_test_set.csv'

#### create simulated graph


In [ ]:
WIKIPEDIA_PAGE_USER_DICT_PATH = 'E:\\Datasets\\WikipediaHE\\ProcessedData\\PageUserDict_2018-01-01_2019-01-01.json'

TRAIN_SET_COMMS_PATH = 'Wikipedia_Link_SQL\\Data\\TrainTestSetsPagesRevisions\\train_set_cats.json'
TEST_SET_COMMS_PATH = 'Wikipedia_Link_SQL\\Data\\TrainTestSetsPagesRevisions\\test_set_cats.json'

In [ ]:
import json
with open(TRAIN_SET_COMMS_PATH, 'r') as file:
    train_subreddits = list(json.load(file).keys())
    
with open(TEST_SET_COMMS_PATH, 'r') as file:
    test_subreddits = list(json.load(file).keys())

In [ ]:
print(len(train_subreddits))
print(len(test_subreddits))

In [ ]:
partite_1_label='Community'
partite_2_label='User'

In [ ]:
reddit_train_sim = WikipediaRevisionsGraphSimulator(categories_pages_links_path=WIKIPEDIA_PAGE_USER_DICT_PATH)

reddit_test_sim = WikipediaRevisionsGraphSimulator(categories_pages_links_path=WIKIPEDIA_PAGE_USER_DICT_PATH)

In [ ]:
#gcolorer = GraphCommunityColorer(REDDIT_NORM_GRAPH, gs.updated_partitions.values(), gs.updated_partitions.values())
#gcolorer.color_node_updated_community_attributes()

In [ ]:
# create bi-partite graph
train_bipartite_graph = reddit_train_sim.generate_bi_partite_graph(
    category_list=train_subreddits,
    partite_1_label=partite_1_label,
    partite_2_label=partite_2_label
)

In [ ]:
# create bi-partite graph
test_bipartite_graph = reddit_test_sim.generate_bi_partite_graph(
    category_list=test_subreddits,
    partite_1_label=partite_1_label,
    partite_2_label=partite_2_label
)

In [ ]:
train_edges_df = reddit_train_sim.bipartite_graph_to_edges_df(
    bipartite_graph=train_bipartite_graph, partite_1_label=partite_1_label, partite_2_label=partite_2_label,
    save_path = NORM_EDGES_PATH, save_csv=True)

In [ ]:
test_edges_df = reddit_test_sim.bipartite_graph_to_edges_df(
    bipartite_graph=test_bipartite_graph, partite_1_label=partite_1_label, partite_2_label=partite_2_label,
    save_path = ANOM_EDGES_PATH, save_csv=True)

In [ ]:
# skip create and reduce functions and load file
norm_edges_df = pd.read_csv(NORM_EDGES_PATH, index_col=0)
anom_edges_df = pd.read_csv(ANOM_EDGES_PATH, index_col=0)

In [ ]:
print(f'Number of rows in norm_edges_df: {len(norm_edges_df)}')
print(f'Number of rows in anom_edges_df: {len(anom_edges_df)}')

In [ ]:
def print_props(gs):
    print(f'Num nodes: {len(gs.BPG.nodes)}')
    print(f'Num edges: {len(gs.BPG.edges)}')
    #print(f'Community sizes:\n\t{[len(gs.partitions[comm]) for comm in gs.partitions]}')

In [ ]:
print_props(reddit_train_sim)

In [ ]:
print_props(reddit_test_sim)

#### create bipartite graph

In [ ]:
bi_partite_graph_creator = BiPartiteGraphCreator(partite_1_label, partite_2_label)

In [ ]:
NORM_BPG = bi_partite_graph_creator.create_graph(
    edges_df = norm_edges_df, edge_source=partite_1_label, edge_target=partite_2_label,
    partite_1_attr_df=None, partite_2_attr_df=None, convert_num_string_to_int=False)

bi_partite_graph_creator.print_bi_partite_properties(NORM_BPG)

In [ ]:
ANOM_BPG = bi_partite_graph_creator.create_graph(
    edges_df = anom_edges_df, edge_source=partite_1_label, edge_target=partite_2_label,
    partite_1_attr_df=None, partite_2_attr_df=None, convert_num_string_to_int=False)

bi_partite_graph_creator.print_bi_partite_properties(ANOM_BPG)

In [ ]:
nx.write_gml(NORM_BPG, NORM_BI_PARTITE_GRAPH_PATH)
nx.write_gml(ANOM_BPG, ANOM_BI_PARTITE_GRAPH_PATH)

#### project graph to one space

In [ ]:
#PG = bi_partite_graph.project_bipartite_to_space(G, 'User', 5)

In [ ]:
# create a subgraph with nodes with at least 1 edge
#PG_subgraph_to_comm_detect=nx.subgraph(PG,[node for node in PG.nodes() if PG.degree(node)>0])

In [ ]:
#nx.write_gml(PG, PROJECTED_BI_PARTITE_GRAPH_2_PATH)

#### sample graph to create tes edges, train positive edges and train negative edges

In [ ]:
train_set_positive_edges, train_set_negative_edges = graph_sampler.sample_single_graph_edges(
    train_graph=NORM_BPG, partite_1_name=partite_1_label, partite_2_name=partite_2_label,
    max_edges=None, negative_edges=True)

In [ ]:
test_set_edges, _ = graph_sampler.sample_single_graph_edges(
    train_graph=ANOM_BPG, partite_1_name=partite_1_label, partite_2_name=partite_2_label,
    max_edges=None, negative_edges=False)

#### Extract topological features - first divide to train and test set and then extract

In [ ]:
train_topological_feature_extractor = TopologicalFeatureExtractor(NORM_BPG)
test_topological_feature_extractor = TopologicalFeatureExtractor(ANOM_BPG)

In [ ]:
train_edges_topological_features_df = train_topological_feature_extractor.create_topological_features_df(
    positive_edges=train_set_positive_edges, negative_edges=train_set_negative_edges,
    save_path=EDGES_TOPOLOGICAL_FEATURES_TRAIN_SET_PATH, save_csv=True)

In [ ]:
test_edges_topological_features_df = test_topological_feature_extractor.create_topological_features_df(
    positive_edges=test_set_edges, negative_edges=None,
    save_path=EDGES_TOPOLOGICAL_FEATURES_TEST_SET_PATH, save_csv=True)

In [ ]:
# skip topological features extraction

train_edges_topological_features_df = pd.read_csv(EDGES_TOPOLOGICAL_FEATURES_TRAIN_SET_PATH, index_col=0)
test_edges_topological_features_df = pd.read_csv(EDGES_TOPOLOGICAL_FEATURES_TEST_SET_PATH, index_col=0)

test_edges_topological_features_df.head()

#### Train a classifier on topological features train set and predict test set

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

In [ ]:
# instantiate topological feature classifier, with a classifier object and train set
topo_feat_clf = TopologicalFeatureClassifier(
    XGBClassifier(), train_set_df=train_edges_topological_features_df, label_column='link_exist')

In [ ]:
# train classifier
topo_feat_clf.fit(val_size=0.1, verbose=True)

In [ ]:
# show predictions in test set
#topo_feat_clf.show_predictions_edges_topological_features_df(df=test_edges_topological_features_df)

topo_feat_clf.test_set_edges_prediction_summary(test_edges_topological_features_df)
topo_feat_clf.print_test_set_edges_prediction_summary()

#### Meta-features extraction

In [ ]:
# get edge existing probabilities (from predictions)
edges_exist_prob_attr_dict = topo_feat_clf.edges_existence_probability_dict(
    df=test_edges_topological_features_df, comm_before_user=True, index_str_before_digit=False)

In [ ]:
len(edges_exist_prob_attr_dict)

In [ ]:
len([val for val in edges_exist_prob_attr_dict.values() if  val])

In [ ]:
meta_feature_extractor = MetaFeatureExtractor(g=ANOM_BPG, edges_existence_prob_dict=edges_exist_prob_attr_dict)

In [ ]:
len(meta_feature_extractor.g.nodes)

In [ ]:
test_set_meta_features_df = meta_feature_extractor.get_all_meta_features(
    thresh=0.8, label=None, csv_save_path=NODES_META_FEATURES_TEST_SET_PATH, save_csv=True)

In [ ]:
test_set_meta_features_df.head()

In [ ]:
# instantiate meta-feature ranker object
meta_feature_ranker = MetaFeatureRanker(test_set_meta_features_df)

In [ ]:
# add column of ranking per feature
meta_feature_ranker.rank_columns()
meta_feature_ranker.ranked_df.head()

In [ ]:
def dd(df, subreddit):
    maxes = [int(x) for x in df.max()[7:-1]]
    sub_rank = [int(x) for x in df.loc[subreddit][7:-1]]
    return list(zip(*(sub_rank, maxes)))
    

In [ ]:
#dd(meta_feature_ranker.ranked_df, 'theblackvoid')

In [ ]:
n=5
bottom_df = meta_feature_ranker.rank_top_bottom_all_features(n=n, top_or_bottom='bottom')
top_df = meta_feature_ranker.rank_top_bottom_all_features(n=n, top_or_bottom='top')

In [ ]:
top_df

In [ ]:
bottom_df

In [ ]:
communities = [node for node, data in ANOM_BPG.nodes(data=True) if data['partite'] == 'Community']

In [ ]:
a = list(ANOM_BPG.degree(communities))
a.sort(key= lambda x: x[1])

In [ ]:
#a

In [ ]:
uniqes = set()

for i, col in enumerate(top_df.columns[2:]):
    if i % 2 == 1:
        continue
    uniqes=uniqes.union((set(top_df[col].to_list())))
    
for i, col in enumerate(bottom_df.columns[2:]):
    if i % 2 == 1:
        continue
    uniqes=uniqes.union((set(bottom_df[col].to_list())))
    
len((uniqes))

In [ ]:
k = 100
largest_smallest = set(list(zip(*(a[:k] + a[-k:])))[0])
len([u for u in uniqes if u not in largest_smallest])

In [ ]:
uniques_not_small_large = ([u for u in uniqes if u not in largest_smallest])
uniques_not_small_large

In [ ]:
print(f'num. unique in largest and smallest: {len([u for u in uniqes if u in largest_smallest])}')

In [ ]:
uniqes

In [ ]:
[f'[{c}](https://he.wikipedia.org/wiki/{c})<br>' for c in uniqes]

[עופר_לוי](https://he.wikipedia.org/wiki/עופר_לוי)<br>',
 '[הורטוס_מוזיקוס](https://he.wikipedia.org/wiki/הורטוס_מוזיקוס)<br>',
 '[השמיניה_(סדרת_טלוויזיה)](https://he.wikipedia.org/wiki/השמיניה_(סדרת_טלוויזיה))<br>',
 '[ליאור_דטאוקר](https://he.wikipedia.org/wiki/ליאור_דטאוקר)<br>',
 '[התזמורת_הפילהרמונית_של_לונדון](https://he.wikipedia.org/wiki/התזמורת_הפילהרמונית_של_לונדון)<br>',
 '[חסידות_נדבורנה](https://he.wikipedia.org/wiki/חסידות_נדבורנה)<br>',
 '[דגש_קל](https://he.wikipedia.org/wiki/דגש_קל)<br>',
 '[היכל_קבוצת_שלמה](https://he.wikipedia.org/wiki/היכל_קבוצת_שלמה)<br>',
 '[מנזר_אנטוניוס_הקדוש_(קופטי)](https://he.wikipedia.org/wiki/מנזר_אנטוניוס_הקדוש_(קופטי))<br>',
 '[תנועת_הנוער_בני_המושבים](https://he.wikipedia.org/wiki/תנועת_הנוער_בני_המושבים)<br>',
 '[דודו_אהרון](https://he.wikipedia.org/wiki/דודו_אהרון)<br>',
 '[סטיבן_הנדרי](https://he.wikipedia.org/wiki/סטיבן_הנדרי)<br>',
 '[ירון_ברלד](https://he.wikipedia.org/wiki/ירון_ברלד)<br>',
 '[מבצר_ברנדונק](https://he.wikipedia.org/wiki/מבצר_ברנדונק)<br>',
 '[ויולה_אורגניסטה](https://he.wikipedia.org/wiki/ויולה_אורגניסטה)<br>',
 '[בובספוג_מכנסמרובע](https://he.wikipedia.org/wiki/בובספוג_מכנסמרובע)<br>',
 "[ג'אד_טראמפ](https://he.wikipedia.org/wiki/ג'אד_טראמפ)<br>",
 '[בייבלייד](https://he.wikipedia.org/wiki/בייבלייד)<br>',
 '[הבנים_והבנות](https://he.wikipedia.org/wiki/הבנים_והבנות)<br>',
 '[יהודא_ליאון_אשכנזי](https://he.wikipedia.org/wiki/יהודא_ליאון_אשכנזי)<br>',
 '[כבידה](https://he.wikipedia.org/wiki/כבידה)<br>',
 '[צביקה_הדר](https://he.wikipedia.org/wiki/צביקה_הדר)<br>',
 "[דילן_או'בריאן](https://he.wikipedia.org/wiki/דילן_או'בריאן)<br>",
 '[שבאבניקים_(סדרת_טלוויזיה)](https://he.wikipedia.org/wiki/שבאבניקים_(סדרת_טלוויזיה))<br>',
 '[נטע_ברזילי](https://he.wikipedia.org/wiki/נטע_ברזילי)<br>',
 '[יותם_קושניר](https://he.wikipedia.org/wiki/יותם_קושניר)<br>',
 '[התזמורת_הסימפונית_של_רשות_השידור_הלאומית](https://he.wikipedia.org/wiki/התזמורת_הסימפונית_של_רשות_השידור_הלאומית)

[אליאנה_תדהר](https://he.wikipedia.org/wiki/אליאנה_תדהר)<br>',
 '[דונלד_טראמפ](https://he.wikipedia.org/wiki/דונלד_טראמפ)<br>',
 '[רבן_יוחנן_בן_זכאי](https://he.wikipedia.org/wiki/רבן_יוחנן_בן_זכאי)<br>',
 '[משפחת_ברזאני](https://he.wikipedia.org/wiki/משפחת_ברזאני)<br>',
 '[פטריופליס](https://he.wikipedia.org/wiki/פטריופליס)<br>',
 '[הישרדות_VIP_עונה_3](https://he.wikipedia.org/wiki/הישרדות_VIP_עונה_3)<br>',
 '[יומני_הערפד](https://he.wikipedia.org/wiki/יומני_הערפד)<br>',
 '[בותריום](https://he.wikipedia.org/wiki/בותריום)<br>',
 '[כחול_לבן_(רשימה)](https://he.wikipedia.org/wiki/כחול_לבן_(רשימה))<br>',
 '[בנציון_נתניהו](https://he.wikipedia.org/wiki/בנציון_נתניהו)<br>',
 '[ישראל_באולימפיאדת_טוקיו_(2020)](https://he.wikipedia.org/wiki/ישראל_באולימפיאדת_טוקיו_(2020))<br>',
 '[פאודה](https://he.wikipedia.org/wiki/פאודה)<br>',
 '[פרסי_ZD](https://he.wikipedia.org/wiki/פרסי_ZD)<br>',
 '[כביש_453](https://he.wikipedia.org/wiki/כביש_453)<br>',
 '[נסים_גרמה](https://he.wikipedia.org/wiki/נסים_גרמה)<br>',
 '[פיליפ_קירקורוב](https://he.wikipedia.org/wiki/פיליפ_קירקורוב)<br>',
 '[אצטדיון_גרין](https://he.wikipedia.org/wiki/אצטדיון_גרין)<br>',
 '[סמיון_גרפמן](https://he.wikipedia.org/wiki/סמיון_גרפמן)<br>',
 '[לוויינים_ישראליים](https://he.wikipedia.org/wiki/לוויינים_ישראליים)<br>',
 '[בלגיה_באירוויזיון](https://he.wikipedia.org/wiki/בלגיה_באירוויזיון)<br>',
 '[ישעיהו](https://he.wikipedia.org/wiki/ישעיהו)<br>',
 '[צביקה_הדר](https://he.wikipedia.org/wiki/צביקה_הדר)<br>',
 '[שוגר_זאזא](https://he.wikipedia.org/wiki/שוגר_זאזא)<br>',
 "[קמנג'ה](https://he.wikipedia.org/wiki/קמנג'ה)<br>",
 '[אחמד_יאסין](https://he.wikipedia.org/wiki/אחמד_יאסין)

[נפוליאון_בונפרטה](https://he.wikipedia.org/wiki/נפוליאון_בונפרטה)<br>',
 '[כביש_79](https://he.wikipedia.org/wiki/כביש_79)<br>',
 '[בני_גנץ](https://he.wikipedia.org/wiki/בני_גנץ)<br>',
 '[כדברא](https://he.wikipedia.org/wiki/כדברא)<br>',
 '[אורה_מורג](https://he.wikipedia.org/wiki/אורה_מורג)<br>',
 '[גלידה](https://he.wikipedia.org/wiki/גלידה)<br>',
 '[עלמה_(מושב)](https://he.wikipedia.org/wiki/עלמה_(מושב))<br>',
 '[נפתלי_בנט](https://he.wikipedia.org/wiki/נפתלי_בנט)<br>',
 '[נמיות_כיס](https://he.wikipedia.org/wiki/נמיות_כיס)<br>',
 '[בראול_סטארס](https://he.wikipedia.org/wiki/בראול_סטארס)<br>',
 '[פוראבר](https://he.wikipedia.org/wiki/פוראבר)<br>',
 '[כישור_(טווייה)](https://he.wikipedia.org/wiki/כישור_(טווייה))<br>',
 '[מארק_סלבי](https://he.wikipedia.org/wiki/מארק_סלבי)<br>',
 '[השפל_הגדול](https://he.wikipedia.org/wiki/השפל_הגדול)<br>',
 '[אריאנה_גרנדה](https://he.wikipedia.org/wiki/אריאנה_גרנדה)<br>',
 '[פוליגמיה](https://he.wikipedia.org/wiki/פוליגמיה)<br>',
 '[אדן_אזאר](https://he.wikipedia.org/wiki/אדן_אזאר)<br>',
 '[ארמנים](https://he.wikipedia.org/wiki/ארמנים)<br>',
 '[כחול](https://he.wikipedia.org/wiki/כחול)<br>',
 '[חנן_בן-ארי](https://he.wikipedia.org/wiki/חנן_בן-ארי)<br>',
 '[גן_החיות_הלימודי_חיפה](https://he.wikipedia.org/wiki/גן_החיות_הלימודי_חיפה)<br>',
 '[להקת_שלוה](https://he.wikipedia.org/wiki/להקת_שלוה)<br>',
 '[הסינפונייטה_הישראלית_באר_שבע](https://he.wikipedia.org/wiki/הסינפונייטה_הישראלית_באר_שבע)<br>',
 '[הצינור](https://he.wikipedia.org/wiki/הצינור)<br>',
 '[אריק_ובנץ](https://he.wikipedia.org/wiki/אריק_ובנץ)<br>',
 '[קריית_יערים](https://he.wikipedia.org/wiki/קריית_יערים)<br>',
 '[רומניה_באירוויזיון](https://he.wikipedia.org/wiki/רומניה_באירוויזיון)<br>',
 '[קיפי_בן_קיפוד](https://he.wikipedia.org/wiki/קיפי_בן_קיפוד)<br>',
 '[מארק_ויליאמס_(שחקן_סנוקר)](https://he.wikipedia.org/wiki/מארק_ויליאמס_(שחקן_סנוקר))<br>',
 '[ואן_פיס](https://he.wikipedia.org/wiki/ואן_פיס)

[שווייץ_באירוויזיון](https://he.wikipedia.org/wiki/שווייץ_באירוויזיון)<br>',
 "[אלטון_ג'ון](https://he.wikipedia.org/wiki/אלטון_ג'ון)<br>",
 '[אנסמבל_מוזיקה_נובה](https://he.wikipedia.org/wiki/אנסמבל_מוזיקה_נובה)<br>',
 '[התזמורת_הסימפונית_ירושלים](https://he.wikipedia.org/wiki/התזמורת_הסימפונית_ירושלים)<br>',
 '[פיודיפאי](https://he.wikipedia.org/wiki/פיודיפאי)<br>',
 "[הפיג'מות_בקפה_קר_סול](https://he.wikipedia.org/wiki/הפיג'מות_בקפה_קר_סול)<br>",
 '[לא_כולל_שירות](https://he.wikipedia.org/wiki/לא_כולל_שירות)<br>',
 '[BTS](https://he.wikipedia.org/wiki/BTS)<br>',
 '[מוקדמות_מונדיאל_2018_-_אירופה](https://he.wikipedia.org/wiki/מוקדמות_מונדיאל_2018_-_אירופה)<br>',
 '[לירן_אביב](https://he.wikipedia.org/wiki/לירן_אביב)<br>',
 '[התזמורת_הסימפונית_חיפה](https://he.wikipedia.org/wiki/התזמורת_הסימפונית_חיפה)<br>',
 '[מוסיכף](https://he.wikipedia.org/wiki/מוסיכף)<br>',
 '[שמואל_אלבז](https://he.wikipedia.org/wiki/שמואל_אלבז)<br>',
 "[קמנג'ה](https://he.wikipedia.org/wiki/קמנג'ה)<br>",
 '[התזמורת_הסימפונית_תל_אביב](https://he.wikipedia.org/wiki/התזמורת_הסימפונית_תל_אביב)<br>',
 '[שלשול](https://he.wikipedia.org/wiki/שלשול)<br>',
 '[יצחק_רבין](https://he.wikipedia.org/wiki/יצחק_רבין)<br>',
 '[ארתור_פרי](https://he.wikipedia.org/wiki/ארתור_פרי)<br>',
 '[ליאור_סושרד](https://he.wikipedia.org/wiki/ליאור_סושרד)<br>',
 '[אלברט_איינשטיין](https://he.wikipedia.org/wiki/אלברט_איינשטיין)<br>',
 '[תזמורת_חיל_האוויר](https://he.wikipedia.org/wiki/תזמורת_חיל_האוויר)<br>',
 '[בייבלייד](https://he.wikipedia.org/wiki/בייבלייד)<br>',
 '[חסידות_קרעטשניף](https://he.wikipedia.org/wiki/חסידות_קרעטשניף)<br>',
 '[מנדי_רודן](https://he.wikipedia.org/wiki/מנדי_רודן)

2017-todat only n-npminor changes

[חנן_בן-ארי](https://he.wikipedia.org/wiki/חנן_בן-ארי)<br>',
 '[יחידת_אגוז](https://he.wikipedia.org/wiki/יחידת_אגוז)<br>',
 '[טוטלי_ספייס](https://he.wikipedia.org/wiki/טוטלי_ספייס)<br>',
 '[התזמורת_הפילהרמונית_של_לונדון](https://he.wikipedia.org/wiki/התזמורת_הפילהרמונית_של_לונדון)<br>',
 '[מפלגת_העבודה_הישראלית](https://he.wikipedia.org/wiki/מפלגת_העבודה_הישראלית)<br>',
 '[תזמורת_NHK_של_טוקיו](https://he.wikipedia.org/wiki/תזמורת_NHK_של_טוקיו)<br>',
 '[צבי_האוזר](https://he.wikipedia.org/wiki/צבי_האוזר)<br>',
 '[יובל_מנדלסון](https://he.wikipedia.org/wiki/יובל_מנדלסון)<br>',
 '[דני_גבע](https://he.wikipedia.org/wiki/דני_גבע)<br>',
 '[שרית_חדד](https://he.wikipedia.org/wiki/שרית_חדד)<br>',
 '[ישראל_כ"ץ_(הליכוד)](https://he.wikipedia.org/wiki/ישראל_כ"ץ_(הליכוד))<br>',
 '[בנות_הזהב_(סדרת_טלוויזיה_ישראלית)](https://he.wikipedia.org/wiki/בנות_הזהב_(סדרת_טלוויזיה_ישראלית))<br>',
 '[הסינפונייטה_של_לונדון](https://he.wikipedia.org/wiki/הסינפונייטה_של_לונדון)<br>',
 '[מכבי_נתניה](https://he.wikipedia.org/wiki/מכבי_נתניה)<br>',
 '[כניסה_להר_הבית_(הלכה)](https://he.wikipedia.org/wiki/כניסה_להר_הבית_(הלכה))<br>',
 '[גיא_אריאלי](https://he.wikipedia.org/wiki/גיא_אריאלי)<br>',
 '[יומני_הערפד](https://he.wikipedia.org/wiki/יומני_הערפד)<br>',
 '[ויולה_אורגניסטה](https://he.wikipedia.org/wiki/ויולה_אורגניסטה)<br>',
 '[אליפות_העולם_בסנוקר](https://he.wikipedia.org/wiki/אליפות_העולם_בסנוקר)<br>',
 '[לוויינים_ישראליים](https://he.wikipedia.org/wiki/לוויינים_ישראליים)<br>',
 "[קמנג'ה](https://he.wikipedia.org/wiki/קמנג'ה)<br>",
 '[איילת_שקד](https://he.wikipedia.org/wiki/איילת_שקד)<br>',
 '[זאב_צעיר_(סדרת_טלוויזיה)](https://he.wikipedia.org/wiki/זאב_צעיר_(סדרת_טלוויזיה))<br>',
 '[אלינור_אהרון](https://he.wikipedia.org/wiki/אלינור_אהרון)<br>',
 '[נחמן_מברסלב](https://he.wikipedia.org/wiki/נחמן_מברסלב)

In [ ]:
len([c for c in test_subreddits if 'stubs' in c])

In [ ]:
6/26

In [ ]:
149/1000

In [ ]:
6/39